[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/giswqs/mapwidget/blob/master/docs/examples/esm.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/giswqs/mapwidget&urlpath=lab/tree/mapwidget/docs/examples/esm.ipynb&branch=master)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/mapwidget/blob/master/docs/examples/esm.ipynb)

In [ ]:
# %pip install mapwidget

In [ ]:
import os
import anywidget
import traitlets

## Leaflet

In [ ]:
class MapWidget(anywidget.AnyWidget):
    _esm = """
    import * as L from "https://esm.sh/leaflet@1.9.3";
    export function render(view) {
    
            let center = view.model.get("center");
            let zoom = view.model.get("zoom");
            let width = view.model.get("width");
            let height = view.model.get("height");
            
            const container = document.createElement("div");     
            container.style.width = width;       
            container.style.height = height;
            
            const map = L.map(container).setView(center, zoom);
            L.tileLayer("https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png", {
                attribution: 'Map data &copy; <a href="https://www.openstreetmap.org/">OpenStreetMap</a> contributors',
                maxZoom: 24,
            }).addTo(map);
            
            view.el.appendChild(container);
        }
    """

    _css = "https://unpkg.com/leaflet@1.9.3/dist/leaflet.css"

    center = traitlets.List([20, 0]).tag(sync=True, o=True)
    zoom = traitlets.Int(2).tag(sync=True, o=True)
    width = traitlets.Unicode("100%").tag(sync=True, o=True)
    height = traitlets.Unicode("600px").tag(sync=True, o=True)


m = MapWidget(center=[20, 0], zoom=2, height='600px')
m

## Maplibre

In [ ]:
class MapWidget(anywidget.AnyWidget):
    _esm = """
    import maplibregl from "https://esm.sh/maplibre-gl@2.4.0";

    export function render(view) {
    
        let center = view.model.get("center");
        center.reverse();
        let zoom = view.model.get("zoom");
        let width = view.model.get("width");
        let height = view.model.get("height");

        const div = document.createElement("div");
        div.style.width = width;
        div.style.height = height;

        const map = new maplibregl.Map({
          container: div,
          style: 'https://demotiles.maplibre.org/style.json', // stylesheet location
          center: center, // starting position [lng, lat]
          zoom: zoom // starting zoom
        });
        view.el.appendChild(div);
    }
    """
    _css = "https://esm.sh/maplibre-gl@2.4.0?css"

    center = traitlets.List([20, 0]).tag(sync=True, o=True)
    zoom = traitlets.Int(2).tag(sync=True, o=True)
    width = traitlets.Unicode("100%").tag(sync=True, o=True)
    height = traitlets.Unicode("600px").tag(sync=True, o=True)


m = MapWidget(center=[20, 0], zoom=2)
m

## Mapbox

In [ ]:
class MapWidget(anywidget.AnyWidget):
    _esm = """
    import mapboxgl from "https://esm.sh/mapbox-gl@2.13.0";

    export function render(view) {
        let center = view.model.get("center");
        center.reverse();
        let zoom = view.model.get("zoom");
        let width = view.model.get("width");
        let height = view.model.get("height");

        const div = document.createElement("div");
        div.style.width = width;
        div.style.height = height;

        let token = view.model.get("token");

        mapboxgl.accessToken = token;
        const map = new mapboxgl.Map({
            container: div,
            style: "mapbox://styles/mapbox/streets-v12",
            center: center,
            zoom: zoom,
        });
        view.el.appendChild(div);
    }

    """
    _css = "https://esm.sh/mapbox-gl@2.13.0?css"

    default_token = os.environ.get('MAPBOX_TOKEN')
    token = traitlets.Unicode(default_token).tag(sync=True)
    center = traitlets.List([20, 0]).tag(sync=True, o=True)
    zoom = traitlets.Int(2).tag(sync=True, o=True)
    width = traitlets.Unicode('100%').tag(sync=True, o=True)
    height = traitlets.Unicode('600px').tag(sync=True, o=True)


m = MapWidget(center=[20, 0], zoom=2)
m

## Cesium

In [ ]:
class MapWidget(anywidget.AnyWidget):
    _esm = """
    function loadScript(src) {
      return new Promise((resolve, reject) => {
        let script = Object.assign(document.createElement("script"), {
          type: "text/javascript",
          async: true,
          src: src,
        });
        script.addEventListener("load", resolve);
        script.addEventListener("error", reject);
        document.body.appendChild(script);
      });
    };
    
    await loadScript("https://cesium.com/downloads/cesiumjs/releases/1.103/Build/Cesium/Cesium.js");
    
    export function render(view) {
    
      let width = view.model.get("width");
      let height = view.model.get("height");
        
      const div = document.createElement("div");
      div.style.width = width;
      div.style.height = height;
      
      Cesium.Ion.defaultAccessToken = view.model.get("token");

      const viewer = new Cesium.Viewer(div, {
        terrainProvider: Cesium.createWorldTerrain()
      });    

      const buildingTileset = viewer.scene.primitives.add(Cesium.createOsmBuildings());   

      viewer.camera.flyTo({
        destination : Cesium.Cartesian3.fromDegrees(-122.4175, 37.655, 400),
        orientation : {
          heading : Cesium.Math.toRadians(0.0),
          pitch : Cesium.Math.toRadians(-15.0),
        }
      });
      
      view.el.appendChild(div);
    }
    """
    # make sure to include styles
    _css = "https://cesium.com/downloads/cesiumjs/releases/1.103/Build/Cesium/Widgets/widgets.css"
    default_token = os.environ.get('CESIUM_TOKEN')
    token = traitlets.Unicode(default_token).tag(sync=True)
    width = traitlets.Unicode('100%').tag(sync=True, o=True)
    height = traitlets.Unicode('600px').tag(sync=True, o=True)


m = MapWidget()
m

## OpenLayers

In [ ]:
class MapWidget(anywidget.AnyWidget):
    _esm = """
    function loadScript(src) {
      return new Promise((resolve, reject) => {
        let script = Object.assign(document.createElement("script"), {
          type: "text/javascript",
          async: true,
          src: src,
        });
        script.addEventListener("load", resolve);
        script.addEventListener("error", reject);
        document.body.appendChild(script);
      });
    };
    
    await loadScript("https://cdn.jsdelivr.net/npm/ol@v7.3.0/dist/ol.js");
    
    export function render(view) {
    
      let center = view.model.get("center");
      center.reverse();
      let zoom = view.model.get("zoom");
      let width = view.model.get("width");
      let height = view.model.get("height");

      const div = document.createElement("div");
      div.style.width = width;
      div.style.height = height;
      
        var map = new ol.Map({
            target: div,
            layers: [
                new ol.layer.Tile({
                    source: new ol.source.OSM()
                })
            ],
            view: new ol.View({
                center: ol.proj.fromLonLat(center),
                zoom: zoom
            })
        });
      view.el.appendChild(div);
    }
    """
    # make sure to include styles
    _css = "https://cdn.jsdelivr.net/npm/ol@v7.3.0/ol.css"
    center = traitlets.List([20, 0]).tag(sync=True, o=True)
    zoom = traitlets.Int(2).tag(sync=True, o=True)
    width = traitlets.Unicode('100%').tag(sync=True, o=True)
    height = traitlets.Unicode('600px').tag(sync=True, o=True)


m = MapWidget(center=[20, 0], zoom=2)
m